# Collaborative Filtering with Neural Networks

In this notebook we will write a matrix factorization model in pytorch to solve a recommendation problem. Then we will write a more general neural model for the same problem.

The MovieLens dataset (ml-latest-small) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 100004 ratings and 1296 tag applications across 9125 movies. https://grouplens.org/datasets/movielens/. To get the data:

`wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip`

## MovieLens dataset

In [3]:
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2022-02-01 08:32:03--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Распознаётся files.grouplens.org (files.grouplens.org)… 128.101.65.152
Подключение к files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 978202 (955K) [application/zip]
Сохранение в: «ml-latest-small.zip»

ml-latest-small.zip 100%[===================>] 955,28K   756KB/s    за 1,3s    

2022-02-01 08:32:05 (756 KB/s) - «ml-latest-small.zip» сохранён [978202/978202]



In [4]:
!unzip ml-latest-small.zip

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [9]:
import pandas as pd
import numpy as np
from pathlib import Path

In [10]:
PATH = Path("./ml-latest-small/")
list(PATH.iterdir())

[PosixPath('ml-latest-small/ratings.csv'),
 PosixPath('ml-latest-small/movies.csv'),
 PosixPath('ml-latest-small/links.csv'),
 PosixPath('ml-latest-small/tags.csv'),
 PosixPath('ml-latest-small/README.txt')]

In [11]:
! head $PATH/ratings.csv

userId,movieId,rating,timestamp
1,1,4.0,964982703
1,3,4.0,964981247
1,6,4.0,964982224
1,47,5.0,964983815
1,50,5.0,964982931
1,70,3.0,964982400
1,101,5.0,964980868
1,110,4.0,964982176
1,151,5.0,964984041


In [12]:
data = pd.read_csv(PATH/"ratings.csv")

In [13]:
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Encoding data
We enconde the data to have contiguous ids for users and movies. You can think about this as a categorical encoding of our two categorical variables userId and movieId.

In [14]:
# split train and validation before encoding
np.random.seed(3)
msk = np.random.rand(len(data)) < 0.8
train = data[msk].copy()
val = data[~msk].copy()

In [15]:
# here is a handy function modified from fast.ai
def proc_col(col, train_col=None):
    """Encodes a pandas column with continous ids. 
    """
    if train_col is not None:
        uniq = train_col.unique()
    else:
        uniq = col.unique()
    name2idx = {o:i for i,o in enumerate(uniq)}
    return name2idx, np.array([name2idx.get(x, -1) for x in col]), len(uniq)

In [16]:
def encode_data(df, train=None):
    """ Encodes rating data with continous user and movie ids. 
    If train is provided, encodes df with the same encoding as train.
    """
    df = df.copy()
    for col_name in ["userId", "movieId"]:
        train_col = None
        if train is not None:
            train_col = train[col_name]
        _,col,_ = proc_col(df[col_name], train_col)
        df[col_name] = col
        df = df[df[col_name] >= 0]
    return df

In [18]:
# encoding the train and validation data
df_train = encode_data(train)
df_val = encode_data(val, train)

## Embedding layer

In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [20]:
# an Embedding module containing 10 user or item embedding size 3
# embedding will be initialized at random
embed = nn.Embedding(10, 3)

In [21]:
# given a list of ids we can "look up" the embedding corresponing to each id
a = torch.LongTensor([[1,2,0,4,5,1]])
embed(a)

tensor([[[ 0.7357, -0.9484,  0.1846],
         [ 2.3105, -0.8450, -0.8085],
         [-0.6357, -1.0900, -0.5103],
         [ 1.0257,  0.0776, -0.7918],
         [ 2.0527, -0.5036,  1.4639],
         [ 0.7357, -0.9484,  0.1846]]], grad_fn=<EmbeddingBackward>)

## Matrix factorization model

In [22]:
class MF(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.user_emb.weight.data.uniform_(0, 0.05)
        self.item_emb.weight.data.uniform_(0, 0.05)
        
    def forward(self, u, v):
        u = self.user_emb(u)
        v = self.item_emb(v)
        return (u*v).sum(1)   

## Debugging MF model

In [23]:
df_t_e

NameError: name 'df_t_e' is not defined

In [24]:
num_users = 7
num_items = 4
emb_size = 3

user_emb = nn.Embedding(num_users, emb_size)
item_emb = nn.Embedding(num_items, emb_size)
users = torch.LongTensor(df_t_e.userId.values)
items = torch.LongTensor(df_t_e.movieId.values)

NameError: name 'df_t_e' is not defined

In [25]:
U = user_emb(users)
V = item_emb(items)

NameError: name 'users' is not defined

In [18]:
U

tensor([[ 0.1547,  0.2277,  0.2442],
        [ 0.1547,  0.2277,  0.2442],
        [ 0.6601,  0.8225, -1.2139],
        [ 0.6601,  0.8225, -1.2139],
        [ 0.1672, -1.2177,  0.1403],
        [ 0.1672, -1.2177,  0.1403],
        [-1.1907, -1.2933, -0.5506],
        [-1.1907, -1.2933, -0.5506],
        [ 0.1938, -0.0683, -0.8493],
        [ 0.1938, -0.0683, -0.8493],
        [ 0.8506, -1.1564,  1.1165],
        [ 0.8639, -2.5148, -0.8391],
        [ 0.8639, -2.5148, -0.8391]])

In [19]:
# element wise multiplication
U*V 

tensor([[-0.1766,  0.2957,  0.4409],
        [ 0.1205,  0.1733,  0.1165],
        [ 0.5143,  0.6258, -0.5793],
        [-0.5603,  0.3582, -0.5370],
        [-0.1909, -1.5812,  0.2533],
        [ 0.1303, -0.9266,  0.0670],
        [ 1.3594, -1.6793, -0.9940],
        [-0.2324,  1.4822,  0.5151],
        [-0.2212, -0.0887, -1.5335],
        [ 0.0378,  0.0783,  0.7947],
        [ 0.1660,  1.3253, -1.0447],
        [ 0.6730, -1.9135, -0.4004],
        [ 0.1686,  2.8820,  0.7851]])

In [20]:
# what we want is a dot product per row
(U*V).sum(1) 

tensor([ 0.5600,  0.4103,  0.5608, -0.7391, -1.5187, -0.7294, -1.3139,
         1.7649, -1.8434,  0.9108,  0.4466, -1.6409,  3.8357])

## Training MF model

In [26]:
num_users = len(df_train.userId.unique())
num_items = len(df_train.movieId.unique())
print(num_users, num_items) 

610 8998


In [27]:
model = MF(num_users, num_items, emb_size=100) # .cuda() if you have a GPU

In [28]:
def train_epocs(model, epochs=10, lr=0.01, wd=0.0, unsqueeze=False):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    model.train()
    for i in range(epochs):
        users = torch.LongTensor(df_train.userId.values) # .cuda()
        items = torch.LongTensor(df_train.movieId.values) #.cuda()
        ratings = torch.FloatTensor(df_train.rating.values) #.cuda()
        if unsqueeze:
            ratings = ratings.unsqueeze(1)
        y_hat = model(users, items)
        loss = F.mse_loss(y_hat, ratings)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(loss.item()) 
    test_loss(model, unsqueeze)

In [34]:
df_train

,userId,movieId,rating,timestamp
0,0,0,4.0,964982703
1,0,1,4.0,964981247
2,0,2,4.0,964982224
3,0,3,5.0,964983815
6,0,4,5.0,964980868
...,...,...,...,...
100828,609,8997,3.5,1493850155
100832,609,2824,5.0,1493850091
100833,609,2825,5.0,1494273047
100834,609,1214,5.0,1493846352


In [29]:
# Here is what unsqueeze does
ratings = torch.FloatTensor(df_train.rating.values)
print(ratings.shape)
ratings = ratings.unsqueeze(1) # .cuda()
print(ratings.shape)

torch.Size([80450])
torch.Size([80450, 1])


In [30]:
def test_loss(model, unsqueeze=False):
    model.eval()
    users = torch.LongTensor(df_val.userId.values) #.cuda()
    items = torch.LongTensor(df_val.movieId.values) #.cuda()
    ratings = torch.FloatTensor(df_val.rating.values) #.cuda()
    if unsqueeze:
        ratings = ratings.unsqueeze(1)
    y_hat = model(users, items)
    loss = F.mse_loss(y_hat, ratings)
    print("test loss %.3f " % loss.item())

In [31]:
train_epocs(model, epochs=10, lr=0.1)

12.911673545837402
4.85158109664917
2.5974793434143066
3.0992088317871094
0.8485954999923706
1.8219674825668335
2.6588215827941895
2.1388769149780273
1.0929958820343018
0.9750269055366516
test loss 1.848 


In [32]:
train_epocs(model, epochs=15, lr=0.01)

1.6404894590377808
1.0032674074172974
0.711554765701294
0.6613120436668396
0.7262927293777466
0.80423903465271
0.843389630317688
0.8347500562667847
0.7921010851860046
0.7364110350608826
0.6866960525512695
0.6550534963607788
0.644588053226471
0.6501421928405762
0.6617523431777954
test loss 0.822 


In [33]:
train_epocs(model, epochs=15, lr=0.01)

0.6696018576622009
0.6311832666397095
0.6391363739967346
0.6144059896469116
0.6053325533866882
0.6139264106750488
0.6119138598442078
0.5970773100852966
0.5849112272262573
0.5830313563346863
0.5842803120613098
0.5795867443084717
0.5690155029296875
0.5586714148521423
0.5523058176040649
test loss 0.759 


## MF with bias

In [29]:
class MF_bias(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100):
        super(MF_bias, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.item_bias = nn.Embedding(num_items, 1)
        self.user_emb.weight.data.uniform_(0,0.05)
        self.item_emb.weight.data.uniform_(0,0.05)
        self.user_bias.weight.data.uniform_(-0.01,0.01)
        self.item_bias.weight.data.uniform_(-0.01,0.01)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.item_emb(v)
        b_u = self.user_bias(u).squeeze()
        b_v = self.item_bias(v).squeeze()
        return (U*V).sum(1) +  b_u  + b_v

In [32]:
model = MF_bias(num_users, num_items, emb_size=100) #.cuda()

In [33]:
train_epocs(model, epochs=10, lr=0.05, wd=1e-5)

13.233644485473633
9.459980964660645
4.618295669555664
1.2266862392425537
2.4537320137023926
3.888521432876587
2.6157896518707275
1.1573508977890015
0.8204843997955322
1.3100122213363647
test loss 2.126 


In [34]:
train_epocs(model, epochs=10, lr=0.01, wd=1e-5)

1.9130752086639404
1.3447301387786865
0.9572998285293579
0.7714419364929199
0.752704381942749
0.8091325759887695
0.8543495535850525
0.8524782657623291
0.8114585876464844
0.7577651739120483
test loss 0.851 


In [35]:
train_epocs(model, epochs=10, lr=0.001, wd=1e-5)

0.7163214087486267
0.7023102045059204
0.6904919147491455
0.6807348728179932
0.6728458404541016
0.6666097044944763
0.6618107557296753
0.6582220792770386
0.6556380391120911
0.6538312435150146
test loss 0.805 


Note that these models are susceptible to weight initialization, optimization algorithm and regularization.

## Neural Network Model

In [76]:
# Note here there is no matrix multiplication, we could potentially make the embeddings of different sizes.
# Here we could get better results by keep playing with regularization.
    
class CollabFNet(nn.Module):
    def __init__(self, num_users, num_items, emb_size=100, n_hidden=10):
        super(CollabFNet, self).__init__()
        self.user_emb = nn.Embedding(num_users, emb_size)
        self.item_emb = nn.Embedding(num_items, emb_size)
        self.lin1 = nn.Linear(emb_size*2, n_hidden)
        self.lin2 = nn.Linear(n_hidden, 1)
        self.drop1 = nn.Dropout(0.1)
        
    def forward(self, u, v):
        U = self.user_emb(u)
        V = self.item_emb(v)
        x = F.relu(torch.cat([U, V], dim=1))
        x = self.drop1(x)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [77]:
model = CollabFNet(num_users, num_items, emb_size=100) #.cuda()

In [78]:
train_epocs(model, epochs=15, lr=0.05, wd=1e-6, unsqueeze=True) 

13.101761817932129
1.957230806350708
1.2605514526367188
1.3381402492523193
1.061022162437439
1.1385098695755005
0.9165319800376892
0.9622549414634705
0.8723138570785522
0.8084518909454346
0.8500765562057495
0.7535637617111206
0.791947603225708
0.7653028964996338
0.7301635146141052
test loss 0.869 


In [79]:
train_epocs(model, epochs=10, lr=0.01, wd=1e-6, unsqueeze=True)

0.7691234350204468
0.9072751402854919
0.7757670879364014
0.7180655598640442
0.7918605208396912
0.7724899053573608
0.7119362950325012
0.7106000185012817
0.7403213977813721
0.7438958883285522
test loss 0.816 


In [80]:
train_epocs(model, epochs=10, lr=0.001, wd=1e-6, unsqueeze=True)

0.7163267731666565
0.7032808065414429
0.695513904094696
0.6967512369155884
0.6998187303543091
0.700666606426239
0.7004959583282471
0.6982167959213257
0.6955875158309937
0.694402813911438
test loss 0.796 


In [81]:
train_epocs(model, epochs=10, lr=0.001, wd=1e-6, unsqueeze=True)

0.6919353008270264
0.6934647560119629
0.6922585368156433
0.6942275762557983
0.6926798224449158
0.6916202902793884
0.6911264061927795
0.6923496127128601
0.6922929286956787
0.6904215812683105
test loss 0.795 


# References
* This notebook is based on [lesson 5 of Jeremy Howard's Deep Learning Course](https://github.com/fastai/fastai/blob/master/courses/dl1/lesson5-movielens.ipynb)